# Notizbuch für den 'data' Endpunkt & Spezifikation von 'job=true'

In [1]:
import requests
from pygenesis.http_helper import get_data_from_endpoint, load_data

Initial wird 'job=false' gesetzt, dies ist identisch zum nicht spezifizieren des Parameteres.

In [2]:
endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0005',
          'area':'all',
          'timeslices':20,
          'sortcriterion': 'Auftragstyp',
          'job':'false',
          'language':'de',}
response = load_data(endpoint=endpoint, method=method, params=params)
#initial_response = response.json()

Die Daten sind zu groß um direkt abgerufen zu werden.Es muss ein Job gestartet werden, der einige Sekunden braucht,um die Daten bereitzustellen und abzurufen.
 Sollen wir einen Job starten?
 Ja/Nein:


{'Ident': {'Service': 'data', 'Method': 'tablefile'}, 'Status': {'Code': 99, 'Content': 'Der Bearbeitungsauftrag wurde erstellt. Die Tabelle kann in Kürze als Ergebnis mit folgendem Namen abgerufen werden: 42151-0005_609791721', 'Type': 'Information'}, 'Parameter': {'username': '********************', 'password': '********************', 'name': '42151-0005', 'area': 'Alle', 'compress': 'false', 'transpose': 'false', 'startyear': '', 'endyear': '', 'timeslices': '20', 'regionalvariable': '', 'regionalkey': '', 'classifyingvariable1': '', 'classifyingkey1': '', 'classifyingvariable2': '', 'classifyingkey2': '', 'classifyingvariable3': '', 'classifyingkey3': '', 'format': 'csv', 'job': 'true', 'stand': '01.01.1970 01:00', 'language': 'de'}, 'Object': None, 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 0, 'Content': 'erfolgreich', 'Type': 'Information'}, 'Parameter': {'username': '********************', '

DestatisStatusError: Die Tabelle ist zu gross, um im Dialog abgerufen zu werden. Bitte nutzen sie den Webservice mit dem Parameter auftrag=true um eine Verarbeitung im Hintergrund zu starten.

In [3]:
endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0005',
          'area':'all',
          'timeslices':20,
          'sortcriterion': 'Auftragstyp',
          'job':'false',
          'language':'de',}
response = get_data_from_endpoint(endpoint=endpoint, method=method, params=params)
#initial_response = response.json()

Die Daten sind zu groß um direkt abgerufen zu werden.Es muss ein Job gestartet werden, der einige Sekunden braucht,um die Daten bereitzustellen und abzurufen.
 Sollen wir einen Job starten?
 Ja/Nein:


{'Ident': {'Service': 'data', 'Method': 'tablefile'}, 'Status': {'Code': 99, 'Content': 'Der Bearbeitungsauftrag wurde erstellt. Die Tabelle kann in Kürze als Ergebnis mit folgendem Namen abgerufen werden: 42151-0005_610144985', 'Type': 'Information'}, 'Parameter': {'username': '********************', 'password': '********************', 'name': '42151-0005', 'area': 'Alle', 'compress': 'false', 'transpose': 'false', 'startyear': '', 'endyear': '', 'timeslices': '20', 'regionalvariable': '', 'regionalkey': '', 'classifyingvariable1': '', 'classifyingkey1': '', 'classifyingvariable2': '', 'classifyingkey2': '', 'classifyingvariable3': '', 'classifyingkey3': '', 'format': 'csv', 'job': 'true', 'stand': '01.01.1970 01:00', 'language': 'de'}, 'Object': None, 'Copyright': '© Statistisches Bundesamt (Destatis), 2022'}
{'Ident': {'Service': 'catalogue', 'Method': 'jobs'}, 'Status': {'Code': 0, 'Content': 'erfolgreich', 'Type': 'Information'}, 'Parameter': {'username': '********************', '

DestatisStatusError: Die Tabelle ist zu gross, um im Dialog abgerufen zu werden. Bitte nutzen sie den Webservice mit dem Parameter auftrag=true um eine Verarbeitung im Hintergrund zu starten.

In [ ]:
response

In [ ]:
initial_response

In [ ]:
initial_response.get('Status')

Der Status ist vom Typ Fehler, da die Tabelle zu groß ist & mittels eines Jobs abgerufen werden muss. Wir stellen somit erneut den gleichen Funktionsaufruf, diese mal aber mit 'job=true'.

In [ ]:
if initial_response.get('Status').get('Code') == 98:
    params |= {'job':'true'}
    response = get_response_from_endpoint(endpoint, method, params)
    jobs_response = response.json()
    print('Der Job wurde gestartet')

In [ ]:
jobs_response

Der Informations-Code 99 zeigt an, dass der Job angestoßen wurde. Die ID des Jobs lässt sich im Content des Status finden.

In [ ]:
if jobs_response.get('Status').get('Code') == 99:
    s = jobs_response.get('Status').get('Content')
    job_ID = s.split(':')[1].strip()
    print('Der Job wurde angestoßen mit der ID: {}'.format(job_ID))

Wir können den Status des Jobs über den 'catalgoue' Endpunkt abfragen.

In [ ]:
params_catalogue = {
    'selection':job_ID,
    'sortcriterion':'date',
    'area':'Öffentlich',
    'pagelength':20,
    'language':'de',
}
catalogue_response = get_response_from_endpoint('catalogue', 'jobs?', params)

In [ ]:
for elem in catalogue_response.json().get('List'):
    if elem.get('Code') == job_ID:
        job_result = elem
        print(job_result)
        print('Der Endpunkt catalogue/jobs wurde angesprochen. Der Status ist: {}'.format(job_result.get('State')))

In [ ]:
if job_result.get('State') != 'Fertig':
    print('Es wird auf die Fertigstellung des Jobs gewartet!'
          '\nBitte teste diesen Block später erneut.')
else:
    if job_ID == job_result.get('Code'):
        params_resultfile = {
            'name':job_ID,
            'area':'all',
            'compress':'false',
            'format':'csv',
            'language':'de',
        }
        resultfile = get_response_from_endpoint('data', 'resultfile?', params)
        resultfile = requests.get('{0}/data/resultfile?username={1}&password={2}&name={3}&area=all&compress=false&format=csv&language=de'.format(genesis_url, genesis_user, genesis_pwd,job_ID))
        print('Das Resultatedokument der ID {0} & wurde in folgender Variable gespeichert \'resultfile\''.format(job_ID))